# sample for scg-symmetry-search

Set the JupyterNotebook kernel by Julia interpreter and execute the following 

In [97]:
using PyCall 
PyCall.python # "~/bin/python3" (python interpreter importing spglib, spinspg, and sympy)
include("./core/mcifparse.jl")

Main.Mcifparse

# Symmetry analysis based on spin space and Shubnikov (magnetic space) group

Please prepare `.mcif` file such as provided in MAGNDATA (http://webbdcrista1.ehu.es/magndata/),
and refer to the file by `magname`


- In SOC-free case, please get symdat by `Mcifparse.get_spinspacegroup`.
- In SOC-coupled case, please get symdat by `Mcifparse.get_shubnikovgroup`.

To check the usage, we recommend to look into the case of CoTa3S6.


## CoTa3S6 / CoNb3S6 : noncoplanar magnet

In [78]:
magname = "examples/CoTa3S6.mcif"
symdat = Mcifparse.get_spinspacegroup(magname)
# symdat = Mcifparse.get_shubnikovgroup(magname)

(Main.Mcifparse.Compute.SpinspaceOperation[Main.Mcifparse.Compute.SpinspaceOperation("", [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.0, 0.0, 0.0], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]), Main.Mcifparse.Compute.SpinspaceOperation("", [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.24999999990868155, 0.43301270586284324, 5.891277845354972e-17], [1.0323780655509868e-14 -0.5773502677119139 0.816496581972626; -0.5773502677119136 -0.6666666683729853 -0.47140452018775336; 0.8164965819726261 -0.47140452018775364 -0.3333333316270253]), Main.Mcifparse.Compute.SpinspaceOperation("", [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.5, 0.0, 2.945638881623285e-17], [-1.0 -1.7769008004369317e-9 -1.256458950307358e-9; -1.7769011273523564e-9 0.3333333346941755 0.9428090411009333; -1.256458487979129e-9 0.9428090411009332 -0.3333333346941752]), Main.Mcifparse.Compute.SpinspaceOperation("", [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [-0.25000000009131845, 0.43301270586284324, 2.945638963731686e-17], [1.08006975116

### Conductivity tensor

The obtained matrix is 3 by 3 given by 
$$ J_a = \sigma_{ab} E_b.$$
$0.5 \chi_{J1:J1} + 0.5 \chi_{J2:J2}$ indicates 
$$ \sigma_{xx} = \sigma_{yy},$$
while $0.5 \chi_{J1:J2} - 0.5 \chi_{J2:J1}$ indicates 
$$ \sigma_{xy} = - \sigma_{yx},$$
that is anomalous Hall effect.


In [47]:
Mcifparse.conductivity(symdat...)

3×3 Matrix{SymPy.Sym}:
  0.5⋅χ_{J1;J1} + 0.5⋅χ_{J2;J2}  0.5⋅χ_{J1;J2} - 0.5⋅χ_{J2;J1}              0
 -0.5⋅χ_{J1;J2} + 0.5⋅χ_{J2;J1}  0.5⋅χ_{J1;J1} + 0.5⋅χ_{J2;J2}              0
                              0                              0  1.0⋅χ_{J3;J3}

### Spin and Orbital magnetization

In [51]:
# uniform spin magnetization
Mcifparse.magnetization_sp(symdat...)

3-element Vector{SymPy.Sym}:
 0
 0
 0

In [53]:
# uniform orbital magnetization
Mcifparse.magnetization_orb(symdat...)

3-element Vector{SymPy.Sym}:
          0
          0
 1.0⋅χ_{O3}

### Spin and Orbital magnetoelectric effect

Coefficients with the superscript of "e" ("m") indicates the T-even (T-odd) part.

In CoTa3S6 with SOC effect, the T-even (electric) such as $\tau_{xx}^e = \tau_{yy}^e$ comes from the noncentrosymmetric crystal structure, and T-odd (magnetic) such as $\tau_{xy}^m = -\tau_{yx}^m$ from the noncoplanar magnetic order.


In [75]:
# spin magnetoelectric effect
Mcifparse.magnetoelectricity_sp(symdat...)

3×3 Matrix{SymPy.Sym}:
 0  0  0
 0  0  0
 0  0  0

3×3 Matrix{SymPy.Sym}:
 0  0  0
 0  0  0
 0  0  0

In [74]:
# orbital magnetoelectric effect
Mcifparse.magnetoelectricity_orb(symdat...)

3×3 Matrix{SymPy.Sym}:
 0.5⋅τ_{O1;E1}__{e} + 0.5⋅τ_{O2;E2}__{e}  …                  0
                                     0                     0
                                     0     1.0⋅τ_{O3;E3}__{e}

3×3 Matrix{SymPy.Sym}:
                                      0  …  0
 -0.5⋅τ_{O1;E2}__{m} + 0.5⋅τ_{O2;E1}__{m}     0
                                      0     0

## Mn3Sn

The `.mcif` file is not prepared in the repository to adhere the irreducibility of open resource provided in Bilbao crystallographic server. 
Please get the corresponding `mcif` file in MAGNDATA (http://webbdcrista1.ehu.es/magndata/index.php?this_label=0.199).
Note that the convention for the domain state is modified in `H. Watanabe et al., arXiv:2307.11560` so as to allow for the weak ferromagnetic moment along the $x$-direction in the SOC-coupled case.

In [92]:
magname = "****" # path for imported .mcif file

opsets, spindim, purespinaxis_args = Mcifparse.get_spinspacegroup(magname);
# opsets, spindim, purespinaxis_args = Mcifparse.get_shubnikovgroup(magname)

spinrot = [
        -1/2 sqrt(3)/2 0 ;
         -sqrt(3)/2 -1/2 0;
         0 0 1;
         ]

opsetsrev = Vector{Mcifparse.SpinspaceOperation}([])
for op in opsets
    push!(opsetsrev,Mcifparse.SpinspaceOperation(op.pgname, round.(spinrot*op.pgmat*inv(spinrot),digits=8),op.trans,round.(spinrot*op.spmat*inv(spinrot),digits=5))) # convert spin order to [100] ferromag

end
symdat = opsetsrev, spindim, purespinaxis_args



(Main.Mcifparse.Compute.SpinspaceOperation[Main.Mcifparse.Compute.SpinspaceOperation("", [1.0 -0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.0, 0.0, 0.0], [1.0 -0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]), Main.Mcifparse.Compute.SpinspaceOperation("", [-1.0 0.0 0.0; -0.0 -1.0 0.0; 0.0 -0.0 -1.0], [2.220446049250313e-16, 3.8459253727671276e-16, 6.799655732577036e-32], [1.0 -0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]), Main.Mcifparse.Compute.SpinspaceOperation("", [0.5 -0.8660254 0.0; 0.8660254 0.5 0.0; -0.0 0.0 1.0], [2.2204460492503128e-16, 0.0, 0.5], [-0.5 -0.86603 0.0; 0.86603 -0.5 0.0; 0.0 0.0 1.0]), Main.Mcifparse.Compute.SpinspaceOperation("", [-0.5 0.8660254 0.0; -0.8660254 -0.5 0.0; 0.0 -0.0 -1.0], [2.4651903288156616e-32, 3.8459253727671276e-16, 0.5], [-0.5 -0.86603 0.0; 0.86603 -0.5 0.0; 0.0 0.0 1.0]), Main.Mcifparse.Compute.SpinspaceOperation("", [-0.5 -0.8660254 0.0; 0.8660254 -0.5 0.0; -0.0 0.0 1.0], [3.330669073875469e-16, 1.9229626863835638e-16, 5.099741799432778e-32], [-0.5 0.86603 0.0; -0.866

### Spin- and Orbital-polarized current induced by electric fields

The coefficient $\tau_{Jasb;Ec}$ denote the response
$$ J_a^{s_b} = \sigma_{ac}^b E_c,$$
where the electric field along $c$ induces the $b$-polarized spin current flowing in $a$-direction.

Similarly, $\tau_{Jaob;Ec}$ is for response with $b$-polarized orbital current.

Response with the superscript "e" ("m") is T-even (E-odd) response. 


In [93]:
# spin case
Mcifparse.E_to_spincurrent(symdat...)

3×3 Matrix{SymPy.Sym}:
 0  0  0
 0  0  0
 0  0  0

3×3 Matrix{SymPy.Sym}:
 0.25*τ^{m}_{J1s1;E1} + 0.25*τ^{m}_{J1s2;E2} - 0.25*τ^{m}_{J2s1;E2} + 0.25*τ^{m}_{J2s2;E1}  …  0
                                                                                         0     0
                                                                                         0     0

3×3 Matrix{SymPy.Sym}:
 0  0  0
 0  0  0
 0  0  0

3×3 Matrix{SymPy.Sym}:
                                                                                         0  …  0
 0.25*τ^{m}_{J1s1;E1} + 0.25*τ^{m}_{J1s2;E2} - 0.25*τ^{m}_{J2s1;E2} + 0.25*τ^{m}_{J2s2;E1}     0
                                                                                         0     0

3×3 Matrix{SymPy.Sym}:
                                          0  …  0
 -0.5⋅τ_{J1s3;E2}__{e} + 0.5⋅τ_{J2s3;E1}__{e}     0
                                          0     0

3×3 Matrix{SymPy.Sym}:
 0  0  0
 0  0  0
 0  0  0

In [94]:
# orbital case
Mcifparse.E_to_orbitalcurrent(symdat...)

3×3 Matrix{SymPy.Sym}:
 0  …                                           0
 0     -0.5⋅τ_{J1o2;E3}__{e} + 0.5⋅τ_{J2o1;E3}__{e}
 0                                              0

3×3 Matrix{SymPy.Sym}:
 0  0  0
 0  0  0
 0  0  0

3×3 Matrix{SymPy.Sym}:
                                          0  …  0.5⋅τ_{J1o2;E3}__{e} - 0.5⋅τ_{J2o1;E3}__{e}
                                          0                                             0
 -0.5⋅τ_{J3o1;E2}__{e} + 0.5⋅τ_{J3o2;E1}__{e}                                             0

3×3 Matrix{SymPy.Sym}:
 0  0  0
 0  0  0
 0  0  0

3×3 Matrix{SymPy.Sym}:
                                          0  …  0
 -0.5⋅τ_{J1o3;E2}__{e} + 0.5⋅τ_{J2o3;E1}__{e}     0
                                          0     0

3×3 Matrix{SymPy.Sym}:
 0  0  0
 0  0  0
 0  0  0

## Cr2O3

In [98]:
magname = "***" # put the path
symdat = Mcifparse.get_spinspacegroup(magname)
# symdat = Mcifparse.get_shubnikovgroup(magname)



(Main.Mcifparse.Compute.SpinspaceOperation[Main.Mcifparse.Compute.SpinspaceOperation("", [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.0, 0.0, 0.0], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]), Main.Mcifparse.Compute.SpinspaceOperation("", [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [1.2952601953960158e-16, 0.5773502691896257, 0.6666666666666666], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]), Main.Mcifparse.Compute.SpinspaceOperation("", [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.5000000000000001, 0.28867513459481287, 0.3333333333333333], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]), Main.Mcifparse.Compute.SpinspaceOperation("", [-1.0 0.0 0.0; 0.0 -1.0 0.0; 0.0 0.0 -1.0], [0.4999999999999997, 0.8660254037844384, 0.9999999999999997], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 -1.0]), Main.Mcifparse.Compute.SpinspaceOperation("", [-1.0 0.0 0.0; 0.0 -1.0 0.0; 0.0 0.0 -1.0], [-3.145631903104611e-16, 0.5773502691896255, 0.6666666666666663], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 -1.0]), Main.Mcifparse.Compute.Spinspa

### Spin magnetoelectricity

In [99]:
# spin magnetoelectric effect
Mcifparse.magnetoelectricity_sp(symdat...)

3×3 Matrix{SymPy.Sym}:
 0  0  0
 0  0  0
 0  0  0

3×3 Matrix{SymPy.Sym}:
 0  0                  0
 0  0                  0
 0  0  1.0⋅τ_{S3;E3}__{m}